In [1]:
from datetime import datetime, date
import warnings
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
pd.options.display.max_columns = None
warnings.filterwarnings('ignore')


## Combine and clean data files from the NSW government

The following code joins all of the CSV files from NSW government into a single CSV.

In [2]:
# All data files from the NSW government can be found at http://maps.six.nsw.gov.au/csv/current/suburb/
# All of the 4325 files are quite large and should be updated, so they are not included in the github repo.

# If a new gov dataset is needed, just re-download all the csv files from the website above and run this cell.
# I used chrono download manager's sniffer feature to download them all.

# I know the analysis is based on Sydney house prices but these government data files are for the whole state of NSW Australia.
# I'm sure there's a smart way to get rid of some of these files to reduce the data size, and improve process time,
# but for now I'm just too lazy to do that.

# uncomment the following code if you want to re-combine the csv files.




# extension = 'csv'
# all_filenames = [i for i in glob.glob('./data/gov/*.{}'.format(extension))]


# combined = pd.concat([pd.read_csv(f) for f in all_filenames ]) #combine all files in the list


# combined = combined.drop(['MULTI-PROPERTY SALE (Y/N)', 'STRATA/NON STRATA', 'PROPERTY NUMBER',
#                'DEALING NUMBER', 'EXTRACTION DATE'], axis=1) # removing less useful columns



# combined = combined.drop_duplicates(keep='last') # removing duplicates


# combined['SALE DATE']=pd.to_datetime(combined['SALE DATE']) # do the hard work now so don't have to do this again each time

# combined.shape


# combined.to_csv( "./data/gov_combined.csv", index=False, encoding='utf-8-sig') #export to csv


In [3]:
# check the shape and columns of the data frame
gov1 = pd.read_csv('./data/gov_combined.csv') # if you don't want to run the prvious cell again every time, just comment it out
gov1 = gov1.sort_values(by="SALE DATE", ascending=False) 
print(gov1.shape)
gov1.head()


(922212, 4)


,ADDRESS,SALE PRICE,SALE DATE,AREA
117973,"323 MICA STREET, BROKEN HILL NSW 2880",105000,2022-02-10,474.3
170642,"6 MACKELLAR STREET, CESSNOCK NSW 2325",43307,2022-02-10,737.7
720892,"30 HORIZONS DRIVE, SALAMANDER BAY NSW 2317",330000,2022-02-10,697.7
70068,"14 GREENWOOD AVENUE, BERKELEY VALE NSW 2261",20000,2022-02-10,833.0
227554,"33 BRIGHT PARADE, DAPTO NSW 2530",270000,2022-02-09,620.5


In [4]:
# Split street address and suburb

addr = gov1.ADDRESS.str.split(
    ',', n=1, expand=True).apply(lambda x: x.str.strip())
gov2 = gov1.copy()
for i in addr.columns:
    gov2[i] = addr[i]


In [5]:
gov2.head()

,ADDRESS,SALE PRICE,SALE DATE,AREA,0,1
117973,"323 MICA STREET, BROKEN HILL NSW 2880",105000,2022-02-10,474.3,323 MICA STREET,BROKEN HILL NSW 2880
170642,"6 MACKELLAR STREET, CESSNOCK NSW 2325",43307,2022-02-10,737.7,6 MACKELLAR STREET,CESSNOCK NSW 2325
720892,"30 HORIZONS DRIVE, SALAMANDER BAY NSW 2317",330000,2022-02-10,697.7,30 HORIZONS DRIVE,SALAMANDER BAY NSW 2317
70068,"14 GREENWOOD AVENUE, BERKELEY VALE NSW 2261",20000,2022-02-10,833.0,14 GREENWOOD AVENUE,BERKELEY VALE NSW 2261
227554,"33 BRIGHT PARADE, DAPTO NSW 2530",270000,2022-02-09,620.5,33 BRIGHT PARADE,DAPTO NSW 2530


In [6]:
gov2.rename(columns={0:'ADDR', 1:'SUBURB', 'SALE DATE':'DATE','SALE PRICE':'PRICE'},inplace=True)
gov2.head()


,ADDRESS,PRICE,DATE,AREA,ADDR,SUBURB
117973,"323 MICA STREET, BROKEN HILL NSW 2880",105000,2022-02-10,474.3,323 MICA STREET,BROKEN HILL NSW 2880
170642,"6 MACKELLAR STREET, CESSNOCK NSW 2325",43307,2022-02-10,737.7,6 MACKELLAR STREET,CESSNOCK NSW 2325
720892,"30 HORIZONS DRIVE, SALAMANDER BAY NSW 2317",330000,2022-02-10,697.7,30 HORIZONS DRIVE,SALAMANDER BAY NSW 2317
70068,"14 GREENWOOD AVENUE, BERKELEY VALE NSW 2261",20000,2022-02-10,833.0,14 GREENWOOD AVENUE,BERKELEY VALE NSW 2261
227554,"33 BRIGHT PARADE, DAPTO NSW 2530",270000,2022-02-09,620.5,33 BRIGHT PARADE,DAPTO NSW 2530


In [7]:
# Leaving only the useful columns

cols = ['ADDR','SUBURB','PRICE','AREA','DATE']
gov3 = gov2[cols]
gov3.columns = ['Address','Suburb','Price','Size','Date']
gov3.head()

,Address,Suburb,Price,Size,Date
117973,323 MICA STREET,BROKEN HILL NSW 2880,105000,474.3,2022-02-10
170642,6 MACKELLAR STREET,CESSNOCK NSW 2325,43307,737.7,2022-02-10
720892,30 HORIZONS DRIVE,SALAMANDER BAY NSW 2317,330000,697.7,2022-02-10
70068,14 GREENWOOD AVENUE,BERKELEY VALE NSW 2261,20000,833.0,2022-02-10
227554,33 BRIGHT PARADE,DAPTO NSW 2530,270000,620.5,2022-02-09


In [8]:
sales_gov = gov3.copy()

## Combine and clean data scraped from RateMyAgent.com.au

In [9]:
# All data files from the RateMyAgent website are scraped with some tool I shall not name.
# The data should be updated once every month.
# This cell is only needed when combining new data with existing data.
# uncomment the following code if you want to re-combine the csv files.




# extension = 'csv'
# all_filenames = [i for i in glob.glob('./data/rma/*.{}'.format(extension))]

# #combine all files in the list
# combined = pd.concat([pd.read_csv(f) for f in all_filenames ])
# #export to csv
# combined.to_csv( "./data/rma_combined.csv", index=False, encoding='utf-8-sig')


In [10]:
rma = pd.read_csv('./data/rma_combined.csv')
rma.shape

(68257, 9)

In [11]:
rma.head()

,Link,Address,Suburb,Beds,Baths,Carpark,Type,Sale Type,Price
0,https://www.ratemyagent.com.au/real-estate-age...,239 Cope St,Waterloo NSW 2017,4.0,2.0,1.0,Townhouse,Sold by Auction on 06 Dec 2021,"$1,495,000"
1,https://www.ratemyagent.com.au/real-estate-age...,6/7 Bank Street,Meadowbank NSW 2114,2.0,1.0,1.0,Unit,Sold by Private Sale on 06 Dec 2021,"$540,000"
2,https://www.ratemyagent.com.au/real-estate-age...,12/1 Ozone Street,Cronulla NSW 2230,2.0,1.0,1.0,Unit,Sold by Auction on 06 Dec 2021,"$1,040,000"
3,https://www.ratemyagent.com.au/real-estate-age...,2/61 Cornelia St,Wiley Park NSW 2195,2.0,1.0,1.0,Unit,Sold by Private Sale on 06 Dec 2021,"$380,000"
4,https://www.ratemyagent.com.au/real-estate-age...,28/570 President Ave,Sutherland NSW 2232,3.0,2.0,2.0,Unit,Sold by Private Sale on 06 Dec 2021,Price unavailable


In [12]:
# remove duplicates

rma1 = rma.drop_duplicates(keep='last')
rma1.shape

(50951, 9)

In [13]:
rma1.isnull().sum()

Link          161
Address         1
Suburb          1
Beds          213
Baths         314
Carpark      4692
Type         1501
Sale Type      18
Price           1
dtype: int64

In [14]:
# Dropping rows that are missing important values

rma1.dropna(subset=['Price','Beds'],inplace=True)
rma1.head()

,Link,Address,Suburb,Beds,Baths,Carpark,Type,Sale Type,Price
0,https://www.ratemyagent.com.au/real-estate-age...,239 Cope St,Waterloo NSW 2017,4.0,2.0,1.0,Townhouse,Sold by Auction on 06 Dec 2021,"$1,495,000"
1,https://www.ratemyagent.com.au/real-estate-age...,6/7 Bank Street,Meadowbank NSW 2114,2.0,1.0,1.0,Unit,Sold by Private Sale on 06 Dec 2021,"$540,000"
2,https://www.ratemyagent.com.au/real-estate-age...,12/1 Ozone Street,Cronulla NSW 2230,2.0,1.0,1.0,Unit,Sold by Auction on 06 Dec 2021,"$1,040,000"
3,https://www.ratemyagent.com.au/real-estate-age...,2/61 Cornelia St,Wiley Park NSW 2195,2.0,1.0,1.0,Unit,Sold by Private Sale on 06 Dec 2021,"$380,000"
4,https://www.ratemyagent.com.au/real-estate-age...,28/570 President Ave,Sutherland NSW 2232,3.0,2.0,2.0,Unit,Sold by Private Sale on 06 Dec 2021,Price unavailable


In [15]:
# Fill null values in column: "Type". If there's a slash(/) in the address, it's an apartment, else it's a house.

nulls = rma1[rma1['Type'].isnull()]
apartments = nulls[nulls.Address.str.contains('/')]
houses = nulls[~nulls.Address.str.contains('/')]
for i in houses.index:
    rma1.at[i,'Type'] = 'House'
for i in apartments.index:
    rma1.at[i,'Type'] = 'Apartment'


In [16]:
# Fill null values in column: "Carpark". If it's an apartment, there's no parking. If it's a house, there's 2 parking spaces.

nulls = rma1[rma1['Carpark'].isnull()]
apartments = nulls[nulls.Address.str.contains('/')]
houses = nulls[~nulls.Address.str.contains('/')]
for i in houses.index:
    rma1.at[i,'Carpark'] = 1
for i in apartments.index:
    rma1.at[i,'Carpark'] = 0

In [17]:
rma1.isnull().sum()

Link         160
Address        0
Suburb         0
Beds           0
Baths        101
Carpark        0
Type           0
Sale Type     17
Price          0
dtype: int64

In [18]:
rma1.head()

,Link,Address,Suburb,Beds,Baths,Carpark,Type,Sale Type,Price
0,https://www.ratemyagent.com.au/real-estate-age...,239 Cope St,Waterloo NSW 2017,4.0,2.0,1.0,Townhouse,Sold by Auction on 06 Dec 2021,"$1,495,000"
1,https://www.ratemyagent.com.au/real-estate-age...,6/7 Bank Street,Meadowbank NSW 2114,2.0,1.0,1.0,Unit,Sold by Private Sale on 06 Dec 2021,"$540,000"
2,https://www.ratemyagent.com.au/real-estate-age...,12/1 Ozone Street,Cronulla NSW 2230,2.0,1.0,1.0,Unit,Sold by Auction on 06 Dec 2021,"$1,040,000"
3,https://www.ratemyagent.com.au/real-estate-age...,2/61 Cornelia St,Wiley Park NSW 2195,2.0,1.0,1.0,Unit,Sold by Private Sale on 06 Dec 2021,"$380,000"
4,https://www.ratemyagent.com.au/real-estate-age...,28/570 President Ave,Sutherland NSW 2232,3.0,2.0,2.0,Unit,Sold by Private Sale on 06 Dec 2021,Price unavailable


In [19]:
# Split sale type and date

rma1['Sale Type'] = rma['Sale Type'].str.replace('Sold by ','' )

In [20]:
saletype = rma1['Sale Type'].str.split(' on ',expand=True)
for i in saletype.columns:
    rma1[i] = saletype[i]

In [21]:
rma2 = rma1.drop('Sale Type',axis=1)
rma2.rename({0:'By',1:'Date'},axis=1,inplace=True)

# trim price
rma2['Price'] = rma2['Price'].str.replace('$','').str.replace(',','')

# to datetime just in case
rma2['Date']=pd.to_datetime(rma2['Date'])



In [22]:
sales_rma = rma2.copy().sort_values(by="Date", ascending=False)
sales_rma

,Link,Address,Suburb,Beds,Baths,Carpark,Type,Price,By,Date
58288,https://www.ratemyagent.com.au/real-estate-age...,7 Lister Place,Rooty Hill NSW 2766,4.0,2.0,2.0,House,Price unavailable,Private Sale,2022-02-15
58274,https://www.ratemyagent.com.au/real-estate-age...,7113 Proposed Road,Box Hill NSW 2765,4.0,2.0,2.0,Development Site,Price unavailable,Private Sale,2022-02-15
58287,https://www.ratemyagent.com.au/real-estate-age...,6 Nicholas Street,Lidcombe NSW 2141,3.0,1.0,2.0,House,Price unavailable,Auction,2022-02-15
58286,https://www.ratemyagent.com.au/real-estate-age...,86 Hastings Street,The Ponds NSW 2769,5.0,2.0,2.0,House,1560000,Private Sale,2022-02-15
58285,https://www.ratemyagent.com.au/real-estate-age...,4/97 The Boulevarde,Wiley Park NSW 2195,2.0,1.0,1.0,Apartment,462000,Private Sale,2022-02-15
...,...,...,...,...,...,...,...,...,...,...
53959,https://www.ratemyagent.com.au/real-estate-age...,145/8-12 Thomas Street,Waitara NSW 2077,2.0,2.0,1.0,Apartment,794000,NaN,NaT
55491,https://www.ratemyagent.com.au/real-estate-age...,225/19-25 Leonard Street,Waitara NSW 2077,2.0,2.0,1.0,Apartment,785000,NaN,NaT
55685,https://www.ratemyagent.com.au/real-estate-age...,ID:21076388/360 Kingsway,Caringbah NSW 2229,3.0,2.0,2.0,Apartment,750000,NaN,NaT
58082,https://www.ratemyagent.com.au/real-estate-age...,95/14-18 Thomas Street,Waitara NSW 2077,2.0,2.0,1.0,Apartment,770000,NaN,NaT


## Prepare datasets before joining

Unify Street Names (Street -> St, etc.)

In [23]:
sales_gov.Address = sales_gov.Address.str.title().str.strip()
sales_gov.Suburb = sales_gov.Suburb.str.title().str.strip()
sales_rma.Address = sales_rma.Address.str.title().str.strip()
sales_rma.Suburb = sales_rma.Suburb.str.title().str.strip()


In [24]:
road_names = sales_rma.Address.str.split(' ', expand = True)


In [25]:
rd_names = road_names[2].unique()

filtered = []
for n in rd_names:
    if n == None:
        pass
    elif len(n)<3:
            filtered.append(n)

filtered


['Pl',
 'St',
 'Rd',
 'Cl',
 'Dr',
 'Ct',
 'La',
 'Ln',
 'Ha',
 'Of',
 'Av',
 'Wy',
 'Cr',
 'Is',
 '4',
 'Sp',
 'On',
 '16',
 'Vw',
 'N',
 '&',
 'De']

In [26]:
road_names[road_names[2]=='Cl']



,0,1,2,3,4,5,6,7,8
58292,7,Jordan,Cl,None,None,None,None,None,None
58622,1,Glenfern,Cl,None,None,None,None,None,None
59299,7,Kay,Cl,None,None,None,None,None,None
59530,4,Staddon,Cl,None,None,None,None,None,None
59367,14/1,Belair,Cl,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
16371,3/8,Dale,Cl,None,None,None,None,None,None
16466,13/7,Hanlon,Cl,None,None,None,None,None,None
16925,12/18,Bowen,Cl,None,None,None,None,None,None
17244,13/18,Bowen,Cl,None,None,None,None,None,None


In [27]:
rma2.loc[237]

Link       https://www.ratemyagent.com.au/real-estate-age...
Address                                  4/61 Portico Parade
Suburb                                   Toongabbie NSW 2146
Beds                                                     4.0
Baths                                                    2.0
Carpark                                                  2.0
Type                                               Townhouse
Price                                                 856000
By                                             Private Sale 
Date                                     2021-11-26 00:00:00
Name: 237, dtype: object

In [28]:
abbr = [
    [' Street',' St'],
    [' Road',' Rd'],
    [' Avenue',' Ave'],
    [' Place',' Pl'],
    [' Close',' Cl'],
    [' Lane',' Ln'],
    [' Drive',' Dr'],
    [' Highway',' Hwy'],
    [' Parade',' Pde'],
    [' Square',' Sq'],
    [' Court',' Ct'],
    [' Glade',' Gld'],

    ['Parkway','Pkwy'],
    ['Boulevard','Blvd'],
    ['Circuit','Cct'],
    
    [' st',' St'],
    [' st.',' St'],
    [' ST',' St'],
    [' RD',' Rd'],
    [' ave', 'Ave'],
    [' AVE', 'Ave'],
    [' road',' Rd'],
    [' ROAD',' Rd'],
    ['Crt','Ct'],
    ['Pde.','Pde'],

    [' Crescent',' Cres'],
    [' Glen',' Gln'],
    [' Plaza', 'Plz'],
    [' View', ' Vw'],

    ['Mallard La', 'Mallard Ln']
]

In [29]:
# replace all street address names with abbreviation
for row in abbr:
    sales_gov.Address = sales_gov.Address.str.replace(row[0],row[1])
    sales_rma.Address = sales_rma.Address.str.replace(row[0],row[1])

In [30]:
sales_gov.sample(5)

,Address,Suburb,Price,Size,Date
913198,71 Horton St,Yagoona Nsw 2199,1157000,706.44,2017-10-28
702976,20 Dover Rd,Rose Bay Nsw 2029,5620000,668.00,2017-09-15
485941,26 Shortland Ave,Lurnea Nsw 2170,725000,556.40,2017-11-27
427898,4/39 Kingscliff St,Kingscliff Nsw 2487,355000,0.00,2017-03-23
256548,7 Finlays Ave,Earlwood Nsw 2206,1745000,520.15,2021-02-20


Unifying Suburb Names:

In [31]:
rma_suburbs = pd.DataFrame(sales_rma.Suburb.unique())
rma_suburbs.columns = ['Name']


In [32]:
gov_suburbs = pd.DataFrame(sales_gov.Suburb.unique())
gov_suburbs.columns = ['Name']

In [33]:
rma_suburbs.Name.isin(gov_suburbs.Name).astype(int).unique()

array([1, 0])

Seems every suburb is included in the NSW dataset, which means there's no error in the Naming

## Join Two Datasets

In [34]:
sales = pd.merge(sales_rma,sales_gov, on=['Address','Suburb'])
sales.shape

(42578, 13)

In [35]:
sales

,Link,Address,Suburb,Beds,Baths,Carpark,Type,Price_x,By,Date_x,Price_y,Size,Date_y
0,https://www.ratemyagent.com.au/real-estate-age...,4/97 The Blvde,Wiley Park Nsw 2195,2.0,1.0,1.0,Apartment,462000,Private Sale,2022-02-15,458000,0.0,2017-05-30
1,https://www.ratemyagent.com.au/real-estate-age...,33/24 Walker St,Rhodes Nsw 2138,2.0,2.0,1.0,Apartment,801000,Private Sale,2022-02-15,801000,0.0,2021-12-22
2,https://www.ratemyagent.com.au/real-estate-age...,2 Lillyana St,Schofields Nsw 2762,4.0,3.0,1.0,House,1065000,Private Sale,2022-02-15,699000,500.2,2018-08-14
3,NaN,21 Dampier Cres,Fairfield West Nsw 2165,3.0,1.0,5.0,House,Price unavailable,Private Sale,2022-02-15,888000,562.7,2022-01-06
4,https://www.ratemyagent.com.au/real-estate-age...,7/1 Fewings St,Clovelly Nsw 2031,2.0,2.0,1.0,Townhouse,Price unavailable,Auction,2022-02-15,1520000,0.0,2017-09-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42573,https://www.ratemyagent.com.au/real-estate-age...,43/344 West Botany St,Brighton-Le-Sands Nsw 2216,4.0,3.0,2.0,Townhouse,1420000,Private Sale,2020-11-03,1420000,0.0,2020-11-03
42574,https://www.ratemyagent.com.au/real-estate-age...,43/344 West Botany St,Brighton-Le-Sands Nsw 2216,4.0,3.0,2.0,Townhouse,1420000,Private Sale,2020-11-03,1402500,0.0,2017-03-16
42575,https://www.ratemyagent.com.au/real-estate-age...,257 The Ponds Blvd,The Ponds Nsw 2769,4.0,2.0,2.0,House,Price unavailable,NaN,NaT,1112400,229.3,2021-07-16
42576,https://www.ratemyagent.com.au/real-estate-age...,1502/3 Network Pl,North Ryde Nsw 2113,1.0,1.0,1.0,Apartment,705000,NaN,NaT,705000,0.0,2021-06-21


In [36]:
# sort by time
sales1=sales.copy()
sales1['Date_y'] = pd.to_datetime(sales['Date_y'])
sales1['Date_x'] = pd.to_datetime(sales['Date_x'])

sales1.sort_values(by='Date_y', ascending=False , inplace=True)
# sales1.set_index(keys='Date_y', drop=True, inplace=True)

In [37]:
sales1.sample(10)

,Link,Address,Suburb,Beds,Baths,Carpark,Type,Price_x,By,Date_x,Price_y,Size,Date_y
5453,https://www.ratemyagent.com.au/real-estate-age...,10 Landor Rd,Barden Ridge Nsw 2234,4.0,3.0,2.0,House,1865000,Auction,2021-12-01,1865000,745.4,2021-12-01
9538,https://www.ratemyagent.com.au/real-estate-age...,1/117 Darley St,Mona Vale Nsw 2103,3.0,1.0,2.0,Townhouse,Price unavailable,Auction,2021-11-12,1800000,0.0,2021-11-05
4217,https://www.ratemyagent.com.au/real-estate-age...,9/230 Arden St,Coogee Nsw 2034,1.0,1.0,1.0,Apartment,Price unavailable,Private Sale,2021-12-09,1150000,0.0,2021-12-10
29148,https://www.ratemyagent.com.au/real-estate-age...,4 Bronte Pl,Woodbine Nsw 2560,3.0,1.0,1.0,House,770000,Private Sale,2021-08-20,770000,723.9,2021-07-30
20555,https://www.ratemyagent.com.au/real-estate-age...,2/58 Pitt St,Mortdale Nsw 2223,2.0,1.0,1.0,Unit,690000,Private Sale,2021-09-30,690000,0.0,2021-09-29
6289,https://www.ratemyagent.com.au/real-estate-age...,26 Tengala Dr,Jordan Springs Nsw 2747,5.0,5.0,2.0,House,Price unavailable,Auction,2021-11-26,1110000,502.2,2021-11-17
20667,https://www.ratemyagent.com.au/real-estate-age...,35C/1 Daunt Ave,Matraville Nsw 2036,1.0,1.0,1.0,Apartment,Price unavailable,Private Sale,2021-09-30,730000,0.0,2021-09-30
10638,https://www.ratemyagent.com.au/real-estate-age...,1E Myrtle St,Loftus Nsw 2232,3.0,1.0,1.0,House,1078000,Auction,2021-11-07,1078000,349.7,2021-11-08
3643,https://www.ratemyagent.com.au/real-estate-age...,29 Kentville Ave,Annandale Nsw 2038,4.0,2.0,1.0,House,2760000,Auction,2021-12-11,2760000,246.6,2021-12-11
26976,https://www.ratemyagent.com.au/real-estate-age...,19 Hovea Pl,Woodford Nsw 2778,4.0,1.0,2.0,House,900000,Private Sale,2021-09-01,900000,1857.0,2021-08-18


In [38]:
cols = ['Address','Suburb','Beds','Baths','Carpark','Type','Size','By','Price_x','Price_y', 'Date_x','Date_y']
sales2 = sales1[cols]

In [39]:
sales2.rename({'Price_x':'Price_rma','Price_y':'Price_gov','Date_x':'Date_rma','Date_y':'Date_gov'},axis=1,inplace=True)
sales2.shape

(42578, 12)

Drop Duplicates

In [40]:
sales2 = sales2.drop_duplicates(subset=['Address','Suburb','Price_gov'],keep='last')

In [41]:
sales2.head(10)

,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price_rma,Price_gov,Date_rma,Date_gov
4365,5 Locksley St,Cronulla Nsw 2230,5.0,2.0,1.0,House,981.25,Auction,Price unavailable,3100000,2021-12-08,2022-02-09
14907,114 Turner Rd,Berowra Heights Nsw 2082,3.0,1.0,2.0,House,709.70,Private Sale,Price unavailable,1540000,2021-10-22,2022-01-31
1920,5 Rocklea Cres,Sylvania Nsw 2224,3.0,1.0,2.0,House,765.10,Auction,Price unavailable,1540000,2021-12-21,2022-01-28
14630,67 Robertson Rd,Bass Hill Nsw 2197,5.0,2.0,3.0,House,1069.00,Auction,1450000,1450000,2021-10-23,2022-01-28
12387,59 Banks Rd,Earlwood Nsw 2206,3.0,2.0,1.0,House,644.90,Private Sale,2100000,2100000,2021-11-01,2022-01-27
2565,8 Marcia St,Toongabbie Nsw 2146,3.0,1.0,3.0,House,1239.30,Auction,1462000,1462000,2021-12-17,2022-01-27
6267,5/6 Bundarra Rd,Bellevue Hill Nsw 2023,3.0,2.0,1.0,House,0.00,Private Sale,Price unavailable,2725000,2021-11-26,2022-01-24
1809,34/1 Werombi Rd,Mount Colah Nsw 2079,2.0,2.0,1.0,Apartment,0.00,Private Sale,Price unavailable,615000,2021-12-21,2022-01-24
1479,85 Cabbage Tree Rd,Bayview Nsw 2104,4.0,2.0,2.0,House,1618.50,Private Sale,3620000,3620000,2021-12-24,2022-01-21
11530,59 Burwood Rd,Concord Nsw 2137,6.0,3.0,4.0,House,999.10,Private Sale,3300000,3300000,2021-11-05,2022-01-21


## Showing the differences between the two datasets

In [42]:
sales_rma[(sales_rma['Suburb']=='Rhodes Nsw 2138')][0:10].drop(['Link','Type','By','Beds','Baths','Carpark'],axis=1)

,Address,Suburb,Price,Date
58283,33/24 Walker St,Rhodes Nsw 2138,801000,2022-02-15
58502,4/9 Llewellyn St,Rhodes Nsw 2138,Price unavailable,2022-02-13
59466,602/7 Rider Blvd,Rhodes Nsw 2138,1190000,2022-02-08
59706,201/16 Sevier Ave,Rhodes Nsw 2138,1270000,2022-02-07
60432,J212/10 Marquet St,Rhodes Nsw 2138,650000,2022-02-03
60623,708/87 Shoreline Dr,Rhodes Nsw 2138,950000,2022-02-02
60787,501/2 Walker St,Rhodes Nsw 2138,Price unavailable,2022-02-01
60664,604/87 Shoreline Dr,Rhodes Nsw 2138,820000,2022-02-01
60659,405/80 Rider Blvd,Rhodes Nsw 2138,680000,2022-02-01
60963,201/43 Shoreline Dr,Rhodes Nsw 2138,Price unavailable,2022-01-31


In [43]:
sales_gov[(sales_gov['Suburb']=='Rhodes Nsw 2138')][0:10].drop(['Size'],axis=1)

,Address,Suburb,Price,Date
692869,1102/87 Shoreline Dr,Rhodes Nsw 2138,1150000,2022-01-30
693813,609/21 Marquet St,Rhodes Nsw 2138,997288,2022-01-14
693057,105/7 Rider Blvd,Rhodes Nsw 2138,1008000,2021-12-31
693324,405/1 Gauthorpe St,Rhodes Nsw 2138,960000,2021-12-30
692974,1510/43 Shoreline Dr,Rhodes Nsw 2138,1000000,2021-12-27
692975,307/43 Shoreline Dr,Rhodes Nsw 2138,650000,2021-12-24
692976,404/43 Shoreline Dr,Rhodes Nsw 2138,650000,2021-12-24
693256,302/63 Shoreline Dr,Rhodes Nsw 2138,818000,2021-12-24
692300,3 Llewellyn St,Rhodes Nsw 2138,45064800,2021-12-23
692299,1 Llewellyn St,Rhodes Nsw 2138,45064800,2021-12-23


In [44]:
sales2[sales2['Suburb']=='Rhodes Nsw 2138'].sort_values('Date_rma',ascending=False)[0:5]

,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price_rma,Price_gov,Date_rma,Date_gov
1,33/24 Walker St,Rhodes Nsw 2138,2.0,2.0,1.0,Apartment,0.0,Private Sale,801000,801000,2022-02-15,2021-12-22
426,708/87 Shoreline Dr,Rhodes Nsw 2138,2.0,2.0,1.0,Apartment,0.0,Private Sale,950000,950000,2022-02-02,2021-12-06
467,501/2 Walker St,Rhodes Nsw 2138,2.0,2.0,1.0,Apartment,0.0,Private Sale,Price unavailable,830000,2022-02-01,2021-12-06
499,201/43 Shoreline Dr,Rhodes Nsw 2138,1.0,1.0,1.0,Apartment,0.0,Private Sale,Price unavailable,675000,2022-01-31,2021-12-20
653,2202/87 Shoreline Dr,Rhodes Nsw 2138,2.0,2.0,1.0,Apartment,0.0,Private Sale,1250080,1250080,2022-01-26,2021-11-12


1. Although RMA's records are newer, sometimes the timestamp comes later than the Gov records. /weird
2. RMA dataset contains some of the sales that's not been submitted to the government. E.g., some of the sales from Sept 2021 are still not in the Gov dataset.
3. RMA dataset is not complete, some of the sales that's in the Gov dataset is not in the RMA dataset. Those properties were probably sold privately or were sold on other platforms.


In [45]:
# This cell exists solely because someone may want to remove the flipped houses (sold again within a few months),
# uncomment the next line if you want one property to have only one sale record, which has the most recent sale record.

# sales3 = sales2.drop_duplicates(subset=['Address','Suburb'],keep='first') 


sales3 = sales2.copy()
sales3.shape

(39718, 12)

## Fill "Price unavailable" from government dataset

The rma data set contains the latest sale prices, but some the prices are listed as "unavailable", either because they are sold else where or because the price was never disclosed by the seller or buyer.

Lucky for us, these transactions have to be disclosed to the government within a few months of selling. So, we can get the missing prices from the government dataset.

** "Last Known Price" Disclaimer

For most of the time, the government dataset lags behind the real time sales, because people need time to settle the deal, sometimes by several months. This is why we can eventually get all of the price data from the government. But for the recent months, we will have to make do with the data from RMA.

One problem caused by filling the price from the government dataset is: if a property was sold again within a few months, the price we fill from the government will be the last known prices, but not the most recent ones.

E.g., 3 month ago, a house was sold by private sale for 1.0 million. Since it's "private", rma didn't get the price (sometimes they do although this time they didn't), so the price of this sale was marked as "unavailable". Then the transaction went through in a few days, and the government got the settled price: "1.0 million" and recorded it. Fast forward to today, the house was sold again for 1.5 million, RMA still didn't get the price(maybe another private sale), so this time it was still marked as "price unavailable". If at this stage we get the data from both rma and government, and fill the price recorded by the government of 1.0 million to this sale, then apparently it's wrong.

In short, this means the data can be misleading, and the house price may not be the actual price, but the "last known price". Fortunately if we update the government dataset constantly in the following months, we will eventually get the accurate prices for recent sales, but of course, the data won't be as fresh by then.

In [46]:
print('There are',sales3[sales3.Price_rma == 'Price unavailable'].shape[0],'records without price.')
sales3[sales3.Price_rma == 'Price unavailable'][0:10]

There are 12320 records without price.


,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price_rma,Price_gov,Date_rma,Date_gov
4365,5 Locksley St,Cronulla Nsw 2230,5.0,2.0,1.0,House,981.25,Auction,Price unavailable,3100000,2021-12-08,2022-02-09
14907,114 Turner Rd,Berowra Heights Nsw 2082,3.0,1.0,2.0,House,709.70,Private Sale,Price unavailable,1540000,2021-10-22,2022-01-31
1920,5 Rocklea Cres,Sylvania Nsw 2224,3.0,1.0,2.0,House,765.10,Auction,Price unavailable,1540000,2021-12-21,2022-01-28
6267,5/6 Bundarra Rd,Bellevue Hill Nsw 2023,3.0,2.0,1.0,House,0.00,Private Sale,Price unavailable,2725000,2021-11-26,2022-01-24
1809,34/1 Werombi Rd,Mount Colah Nsw 2079,2.0,2.0,1.0,Apartment,0.00,Private Sale,Price unavailable,615000,2021-12-21,2022-01-24
845,45 Banbal Rd,Engadine Nsw 2233,5.0,2.0,1.0,House,537.50,Private Sale,Price unavailable,1450000,2022-01-19,2022-01-20
640,25/23 Taranto Rd,Marsfield Nsw 2122,2.0,1.0,1.0,Townhouse,0.00,Private Sale,Price unavailable,885000,2022-01-27,2022-01-20
574,1226/45 Macquarie St,Parramatta Nsw 2150,1.0,NaN,0.0,Studio,0.00,Private Sale,Price unavailable,380000,2022-01-28,2022-01-20
10446,8 Manns Ave,Greenwich Nsw 2065,4.0,2.0,1.0,House,480.00,Auction,Price unavailable,4800000,2021-11-09,2022-01-19
895,45 Cove St,Birchgrove Nsw 2041,3.0,3.0,2.0,House,167.50,Private Sale,Price unavailable,3280000,2022-01-18,2022-01-19


In [47]:
# from dateutil.relativedelta import *

for i in sales3.index:
    entry = sales3.loc[i]
    if entry.Price_rma == 'Price unavailable':
        sales3.at[i, 'Price_rma']=sales3.loc[i].Price_gov

In [48]:
sales3.shape

(39718, 12)

# Clean up

In [49]:
sales3.isnull().sum()[sales3.isnull().sum()!=0]

Baths       49
Size         7
By           3
Date_rma     3
dtype: int64

This means there are entries without bedroom number, size, By, or Date. tbh only the bedroom number is important.

In [50]:
# check for null values in time columns
sales3[sales3.Date_rma.isnull() | sales3.Date_gov.isnull()]

,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price_rma,Price_gov,Date_rma,Date_gov
42577,42 Camberwell Rd,Vineyard Nsw 2765,3.0,1.0,1.0,House,3338.4,NaN,1920000,1920000,NaT,2021-10-28
42575,257 The Ponds Blvd,The Ponds Nsw 2769,4.0,2.0,2.0,House,229.3,NaN,1112400,1112400,NaT,2021-07-16
42576,1502/3 Network Pl,North Ryde Nsw 2113,1.0,1.0,1.0,Apartment,0.0,NaN,705000,705000,NaT,2021-06-21


In [51]:
for i in range(0, len(sales3)):
    if pd.isnull(sales3['Date_rma'].iloc[i]):
        sales3['Date_rma'].iloc[i] = sales3['Date_gov'].iloc[i]


In [52]:
# properties without baths
sales3[sales3.Baths.isnull()]


,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price_rma,Price_gov,Date_rma,Date_gov
574,1226/45 Macquarie St,Parramatta Nsw 2150,1.0,NaN,0.0,Studio,0.0,Private Sale,380000,380000,2022-01-28,2022-01-20
1866,701/79 Oxford St,Bondi Junction Nsw 2022,1.0,NaN,0.0,Studio,0.0,Private Sale,450000,450000,2021-12-21,2021-12-21
2218,23/6 Greenknowe Ave,Elizabeth Bay Nsw 2011,1.0,NaN,0.0,Studio,0.0,Private Sale,680000,680000,2021-12-19,2021-12-20
2583,35/101 Macleay St,Potts Point Nsw 2011,1.0,NaN,0.0,Studio,0.0,Private Sale,383000,383000,2021-12-17,2021-12-17
13660,1/23 Abercrombie St,Chippendale Nsw 2008,1.0,NaN,0.0,Apartment,0.0,Auction,780000,780000,2021-10-28,2021-12-13
3773,203/152 Campbell Pde,Bondi Beach Nsw 2026,1.0,NaN,0.0,Studio,0.0,Private Sale,1010000,1010000,2021-12-10,2021-12-11
4055,11/12 Enmore Rd,Newtown Nsw 2042,1.0,NaN,0.0,Studio,0.0,Private Sale,430000,430000,2021-12-09,2021-12-09
4544,13/4 Mcdonald St,Potts Point Nsw 2011,1.0,NaN,0.0,Apartment,0.0,Private Sale,535000,535000,2021-12-07,2021-12-07
4580,610/187 Kent St,Millers Point Nsw 2000,1.0,NaN,0.0,Apartment,0.0,Private Sale,650000,650000,2021-12-07,2021-11-24
6309,30/14 Ward Ave,Rushcutters Bay Nsw 2011,1.0,NaN,0.0,Apartment,0.0,Private Sale,470000,470000,2021-11-26,2021-11-19


In [53]:
# after checking some of the apartment listed here on other platforms such as domain.com.au,
# I found that most of these 1 bed apartments have at lease 1 bathroom, so they are filled with 1.

sales3.loc[(sales3.Baths.isnull()), 'Baths'] = 1


In [54]:
sales3[sales3.isnull().any(axis=1)]


,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price_rma,Price_gov,Date_rma,Date_gov
42577,42 Camberwell Rd,Vineyard Nsw 2765,3.0,1.0,1.0,House,3338.4,NaN,1920000,1920000,2021-10-28,2021-10-28
16226,13 Hinton St,Spring Farm Nsw 2570,3.0,2.0,1.0,House,NaN,Private Sale,761000,761000,2021-10-18,2021-10-18
800,47A Chalder St,Marrickville Nsw 2204,2.0,1.0,2.0,Warehouse,NaN,Private Sale,2100000,3971000,2022-01-20,2021-09-28
27201,27B Alfred Rd,Forest Lodge Nsw 2037,2.0,2.0,2.0,House,NaN,Auction,2085000,2085000,2021-08-31,2021-08-31
30197,5 Burnell St,Drummoyne Nsw 2047,4.0,2.0,4.0,House,NaN,Auction,7575000,7575000,2021-08-14,2021-08-14
42575,257 The Ponds Blvd,The Ponds Nsw 2769,4.0,2.0,2.0,House,229.3,NaN,1112400,1112400,2021-07-16,2021-07-16
42576,1502/3 Network Pl,North Ryde Nsw 2113,1.0,1.0,1.0,Apartment,0.0,NaN,705000,705000,2021-06-21,2021-06-21
26611,23 Blake St,Rose Bay Nsw 2029,5.0,4.0,2.0,House,NaN,Auction,6197,6197,2021-09-02,2021-05-19
16227,13 Hinton St,Spring Farm Nsw 2570,3.0,2.0,1.0,House,NaN,Private Sale,761000,589950,2021-10-18,2018-08-13
868,1/149 Malabar Rd,South Coogee Nsw 2034,2.0,2.0,1.0,Apartment,NaN,Private Sale,5400000,5400000,2022-01-18,2017-12-20


It doesn't matter to drop these rows.

In [55]:
print("before:", sales3.shape)
sales4 = sales3.dropna()
print("after:",sales4.shape)

before: (39718, 12)
after: (39708, 12)


## Final Trim

In [56]:
recent_sales=sales4.drop(columns=['Price_gov','Date_gov']) # we will use the more recent price column, which was rma's with prices filled from gov's
recent_sales.rename({'Price_rma':'Price','Date_rma':'Date'},axis=1,inplace=True) # we will use only one time column, doesn't matter if it's rma's or gov's
recent_sales.sort_values('Date',ascending=False,inplace=True)
recent_sales = recent_sales.drop_duplicates(subset=["Address","Suburb"]) #keep the latest sales records

#export to csv
recent_sales.to_csv( "./data/recent_sales.csv", index=False, encoding='utf-8-sig')


In [57]:
recent_sales

,Address,Suburb,Beds,Baths,Carpark,Type,Size,By,Price,Date
2,2 Lillyana St,Schofields Nsw 2762,4.0,3.0,1.0,House,500.2,Private Sale,1065000,2022-02-15
1,33/24 Walker St,Rhodes Nsw 2138,2.0,2.0,1.0,Apartment,0.0,Private Sale,801000,2022-02-15
4,7/1 Fewings St,Clovelly Nsw 2031,2.0,2.0,1.0,Townhouse,0.0,Auction,1520000,2022-02-15
5,127/25 Best St,Lane Cove Nsw 2066,3.0,1.0,1.0,Apartment,0.0,Private Sale,1450000,2022-02-15
0,4/97 The Blvde,Wiley Park Nsw 2195,2.0,1.0,1.0,Apartment,0.0,Private Sale,462000,2022-02-15
...,...,...,...,...,...,...,...,...,...,...
42559,6/3 Pine St,Rozelle Nsw 2039,3.0,1.0,1.0,Townhouse,0.0,Auction,1285000,2020-11-03
42560,4/58 Broughton St,Mortdale Nsw 2223,3.0,1.0,2.0,Townhouse,0.0,Private Sale,815000,2020-11-03
42567,5/22 Ramona St,Quakers Hill Nsw 2763,4.0,2.0,2.0,Townhouse,0.0,Private Sale,750000,2020-11-03
42572,5 Kirkham Rd,Auburn Nsw 2144,3.0,3.0,2.0,Townhouse,0.0,Private Sale,718000,2020-11-03


In [58]:
%store recent_sales

Stored 'recent_sales' (DataFrame)
